In [1]:

# Set up notebook
%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')
from FRVRS import nu, fu
nu.delete_ipynb_checkpoint_folders()

Pretty printing has been turned OFF


In [27]:

from pandas import Series
lines_str = r""""""
Series(lines_str.split('\n')).value_counts().head(10)

Attempt to read CSV file using pandas; if unsuccessful, try using a reader                                           51
Set the MCIVR metrics types                                                                                          51
Section off player actions by session start and end                                                                  51
[nan]                                                                                                                13
(199476, 126)                                                                                                         5
Attempting to load /mnt/c/Users/DaveBabbitt/Downloads/saves/pkl/metrics_evaluation_open_world_csv_stats_df.pkl.                 4
                record_count                                                                                          4
Attempting to load /mnt/c/Users/DaveBabbitt/Downloads/saves/pkl/metrics_evaluation_open_world_scene_stats_df.pkl.     3
                  record_count

In [3]:

from FRVRS.notebook_utils import NotebookUtilities
nu = NotebookUtilities(
    data_folder_path='/mnt/c/Users/DaveBabbitt/Downloads/data',
    saves_folder_path='/mnt/c/Users/DaveBabbitt/Downloads/saves'
)

from FRVRS.frvrs_utils import FRVRSUtilities
fu = FRVRSUtilities(
    data_folder_path='/mnt/c/Users/DaveBabbitt/Downloads/data',
    saves_folder_path='/mnt/c/Users/DaveBabbitt/Downloads/saves'
)

In [24]:

# load data frames
data_frames_dict = nu.load_data_frames(
    metrics_evaluation_open_world_csv_stats_df='', metrics_evaluation_open_world_json_stats_df='',
    metrics_evaluation_open_world_scene_stats_df='', metrics_evaluation_open_world_anova_df=''
)
metrics_evaluation_open_world_csv_stats_df = data_frames_dict['metrics_evaluation_open_world_csv_stats_df']
metrics_evaluation_open_world_json_stats_df = data_frames_dict['metrics_evaluation_open_world_json_stats_df']
metrics_evaluation_open_world_scene_stats_df = data_frames_dict['metrics_evaluation_open_world_scene_stats_df']
metrics_evaluation_open_world_anova_df = data_frames_dict['metrics_evaluation_open_world_anova_df']

Attempting to load /mnt/c/Users/DaveBabbitt/Downloads/saves/pkl/metrics_evaluation_open_world_csv_stats_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Downloads/saves/pkl/metrics_evaluation_open_world_json_stats_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Downloads/saves/pkl/metrics_evaluation_open_world_scene_stats_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Downloads/saves/pkl/metrics_evaluation_open_world_anova_df.pkl.


In [10]:

from os import listdir as listdir, makedirs as makedirs, path as osp, remove as remove, sep as sep, walk as walk
from pandas import CategoricalDtype, DataFrame, Index, NaT, Series, concat, isna, notnull, read_csv, read_excel, read_pickle, to_datetime, to_numeric
import re

print("Get column and value descriptions")
file_path = osp.join(fu.data_folder, 'xlsx', 'Metrics_Evaluation_Dataset_organization_for_BBAI.xlsx')
dataset_organization_df = read_excel(file_path)

print("Fix the doubled up descriptions")
mask_series = dataset_organization_df.Labels.map(lambda x: ';' in str(x))
for row_index, label in dataset_organization_df[mask_series].Labels.items():
    labels_list = re.split(' *; *', str(label), 0)
    dataset_organization_df.loc[row_index, 'Labels'] = labels_list[0]
    
    # Get a copy of the row
    new_row = dataset_organization_df.loc[row_index].copy()
    
    # Modify the desired column value
    new_row['Labels'] = labels_list[1]
    
    print("Append the new row to the DataFrame")
    dataset_organization_df = concat([dataset_organization_df, new_row], ignore_index=True)

print("Get a copy of the row")
mask_series = (dataset_organization_df.Variable == 'AD_Del_Omni')
new_row = dataset_organization_df.loc[mask_series].copy()

print("Modify the desired column value")
new_row['Variable'] = 'AD_Del_Omni_Text'

print("Append the new row to the DataFrame")
dataset_organization_df = concat([dataset_organization_df, new_row], ignore_index=True)

# Get the column value descriptions
mask_series = ~dataset_organization_df.Description.isnull()
df = dataset_organization_df[mask_series]
value_description_dict = df.set_index('Variable').Description.to_dict()
new_description_dict = value_description_dict.copy()
for k, v in value_description_dict.items():
    new_description_dict[k] = v
    if (not k.endswith('_Text')):
        new_key_name = f'{k}_Text'
        new_description_dict[new_key_name] = new_description_dict.get(new_key_name, v)
value_description_dict = new_description_dict.copy()

numeric_categories_mask_series = dataset_organization_df.Labels.map(lambda x: '=' in str(x))
value_descriptions_columns = dataset_organization_df[numeric_categories_mask_series].Variable.unique().tolist()
def get_value_description(column_name, column_value):
    value_description = ''
    if not isna(column_value):
        mask_series = (dataset_organization_df.Variable == column_name) & ~dataset_organization_df.Labels.isnull()
        if mask_series.any():
            df = dataset_organization_df[mask_series]
            mask_series = df.Labels.map(lambda label: re.split(' *= *', str(label), 0)[0] == str(int(float(column_value))))
            if mask_series.any():
                label = df[mask_series].Labels.squeeze()
                value_description = re.split(' *= *', str(label), 0)[1]
    
    return value_description

Get column and value descriptions
Fix the doubled up descriptions
Append the new row to the DataFrame
Get a copy of the row
Modify the desired column value
Append the new row to the DataFrame


In [19]:

from numpy import nan, isnan

# Add medical role back in
new_column = 'MedRole'
column_name = 'medical_role'
if new_column in metrics_evaluation_open_world_json_stats_df.columns:
    on_columns = sorted(set(metrics_evaluation_open_world_anova_df.columns).intersection(set(metrics_evaluation_open_world_json_stats_df.columns)))
    columns_list = on_columns + [new_column]
    metrics_evaluation_open_world_anova_df = metrics_evaluation_open_world_anova_df.merge(
        metrics_evaluation_open_world_json_stats_df[columns_list], on=on_columns, how='left'
    ).rename(columns={new_column: column_name})
    metrics_evaluation_open_world_anova_df[column_name] = metrics_evaluation_open_world_anova_df[column_name].map(
        lambda cv: get_value_description('MedRole', cv)
    ).replace('', nan)
print(metrics_evaluation_open_world_anova_df.groupby(column_name).size().to_frame().rename(columns={0: 'record_count'}).sort_values(
    'record_count', ascending=False
).head(5))

                 record_count
medical_role                 
Other                      32
Medical student            17
EM faculty                  6
Paramedic                   5
EM resident                 2


In [25]:

# Add the sim environment back in
new_column = 'encounter_layout'
if new_column in metrics_evaluation_open_world_json_stats_df.columns:
    on_columns = sorted(set(metrics_evaluation_open_world_anova_df.columns).intersection(set(metrics_evaluation_open_world_json_stats_df.columns)))
    columns_list = on_columns + [new_column]
    metrics_evaluation_open_world_anova_df = metrics_evaluation_open_world_anova_df.merge(
        metrics_evaluation_open_world_json_stats_df[columns_list], on=on_columns, how='left'
    )
print(metrics_evaluation_open_world_anova_df.groupby(new_column).size().to_frame().rename(columns={0: 'record_count'}).sort_values(
    'record_count', ascending=False
).head(5))

                  record_count
encounter_layout              
Desert                      20
Submarine                   18
Jungle                      17
Urban                       13


In [2]:

# load libraries
from datetime import timedelta
from pandas import DataFrame
import humanize
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as osp
import pandas as pd
import re

In [3]:

print([f'os.{fn}' for fn in dir(os) if 'dir' in fn.lower()])
print([f'osp.{fn}' for fn in dir(osp) if 'dir' in fn.lower()])
print([f'nu.{fn}' for fn in dir(nu) if 'github' in fn])

['os.DirEntry', 'os.O_DIRECT', 'os.O_DIRECTORY', 'os.ST_NODIRATIME', 'os.chdir', 'os.curdir', 'os.fchdir', 'os.listdir', 'os.makedirs', 'os.mkdir', 'os.pardir', 'os.removedirs', 'os.rmdir', 'os.scandir', 'os.supports_dir_fd']
['osp.curdir', 'osp.dirname', 'osp.isdir', 'osp.pardir']
['nu.github_folder']


In [4]:

print([f'os.{fn}' for fn in dir(os) if 'name' in fn.lower()])
print([f'osp.{fn}' for fn in dir(osp) if 'name' in fn.lower()])
print([f'nu.{fn}' for fn in dir(nu) if 'name' in fn])

['os.__name__', 'os.confstr_names', 'os.name', 'os.pathconf_names', 'os.rename', 'os.renames', 'os.sysconf_names', 'os.ttyname', 'os.uname', 'os.uname_result']
['osp.__name__', 'osp.basename', 'osp.dirname', 'osp.supports_unicode_filenames']
['nu.get_filename_from_url']


In [5]:

os.listdir(path=None)

['Attic.ipynb', 'Installs.ipynb', 'OS Path Navigation.ipynb']

In [16]:

for sub_directory, directories_list, files_list in os.walk(nu.github_folder):
    file_names_list = []
    for file_name in files_list:
        if file_name.endswith('.ipynb') and re.search(r'(\bOW\b|Open(_|%20| )World)', file_name):
            file_names_list.append(file_name)
    if file_names_list:
        print()
        print(sub_directory.split('/')[-1])
        print('\n'.join(file_names_list))


open_world
Analyze OW Patient Stats.ipynb
Correct SORT Order Metric for Metrics Evaluation Open World.ipynb
Data Fixes for Metrics Evaluation Open World.ipynb
Dataset Built for Metrics Evaluation Open World.ipynb
Examine Freeform Open World ADM JSON Files.ipynb
Exploratory Analysis of Open World ITM Data.ipynb
Explore OW Participants and Scenes.ipynb
File Stats Created for Metrics Evaluation Open World.ipynb
Scene Stats Created for Metrics Evaluation Open World.ipynb


In [ ]:

# Define the starting directory
root_dir = '/'

# Use a recursive function to search for matching folders
def find_python311_folders(directory):
    found_folders = []
    try: entries_list = os.scandir(directory)
    except: entries_list = []
    for entry in entries_list:
        try: is_entry_dir = entry.is_dir()
        except: is_entry_dir = False
        if is_entry_dir:
            
            # Check if the current entry is the target folder
            if entry.name == "python3.11": found_folders.append(os.path.join(directory, entry.name))
            
            # Recursively search subdirectories (except the target folder)
            elif entry.name != "python3.11": found_folders.extend(find_python311_folders(os.path.join(directory, entry.name)))
    
    return found_folders

# Start the search from the root directory
python311_folders = find_python311_folders(root_dir)

In [ ]:

# Print the found folders
print(f"Found python3.11 folders: {', '.join(python311_folders)}")

In [ ]:

# Print out a section of the .gitignore
black_list = ['.ipynb_checkpoints', '$Recycle.Bin', '.git']
print()
print('# exclude everything except personal directories')
print('/*')
print('!.gitignore')
for sub_directory in os.listdir(nu.github_folder):
    if all(map(lambda x: x not in sub_directory, black_list)) and osp.isdir(osp.join(nu.github_folder, sub_directory)): print(f'!/{sub_directory}')
print('!README.md')

In [ ]:

columns_set = set()
for pickle_name in [fn.split('.')[0] for fn in os.listdir(s.saves_pickle_folder)]:
    if pickle_name.endswith('_df'):
        df = nu.load_object(pickle_name)
        columns_set.update(df.columns[list(map(lambda x: ('year' in x) and (('begin' in x) or ('end' in x)), df.columns))])
columns_set

In [ ]:

# 'Intel64 Family 6 Model 60 Stepping 3, GenuineIntel'
import platform

platform.processor()